In [1]:
#TODO: add shell commands

In [12]:
#imports
from kafka import KafkaConsumer
from kafka import TopicPartition
import psycopg2
import pandas as pd
import json

In [3]:
#set parameters
kafka_server = 'localhost:9092'

In [4]:
#init consumer
consumer = KafkaConsumer(
    bootstrap_servers=kafka_server,
    auto_offset_reset='earliest',
    value_deserializer=lambda v: json.loads(v.decode('utf-8').replace("'", '"'))
)

In [13]:
#init database
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="postgres")
cursor = conn.cursor()
cursor.execute("select version()")
data = cursor.fetchone()
print("Connection established to: ",data)

Connection established to:  ('PostgreSQL 16.0 (Debian 16.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',)


In [5]:
#find current lenght of 'delays' topic
partition_delays = TopicPartition('delays',0)
consumer.assign([partition_delays])

consumer.seek_to_end(partition_delays)
lastOffset = consumer.position(partition_delays)

consumer.seek_to_beginning(partition_delays)

In [6]:
#consume delays
delay_df = pd.DataFrame()

for msg in consumer:
    row=msg.value
    #currently creating a df, TODO: write to db here
    delay_df = pd.concat([delay_df, pd.DataFrame([row])], ignore_index=True)
    if msg.offset == lastOffset - 1:
            break

In [7]:
#find current lenght of 'weather' topic
partition_weather = TopicPartition('weather',0)
consumer.assign([partition_weather])

consumer.seek_to_end(partition_weather)
lastOffset = consumer.position(partition_weather)

consumer.seek_to_beginning(partition_weather)

In [8]:
#consume weather
weather_df = pd.DataFrame()

for msg in consumer:
    row=msg.value
    #currently creating a df, TODO: write to db here
    weather_df = pd.concat([weather_df, pd.DataFrame([row])], ignore_index=True)
    if msg.offset == lastOffset - 1:
            break

In [9]:
consumer.close()

In [10]:
delay_df.head()

,id,title,lines,stations,start,end
0,163706,Verspätungen,[U3],"[Enkplatz, Erdberg, Gasometer, Herrengasse, Hü...",10.01.2024 18:25,10.01.2024 23:59
1,163705,Polizeieinsatz,"[6, 11, 69A]","[ Zinnergasse, Absberggasse, Alfred-Adler-Stra...",10.01.2024 17:33,10.01.2024 23:59
2,163704,Rettungseinsatz,"[2, 10]","[ Hummelgasse, Albertgasse, Am Tabor, Blindeng...",10.01.2024 17:02,10.01.2024 23:59
3,163703,Rettungseinsatz,[U6],"[Alser Straße, Alterlaa, Am Schöpfwerk, Burgga...",10.01.2024 16:46,10.01.2024 23:59
4,163702,Falschparker,[13A],"[Argentinierstraße, Belvederegasse, Burggasse/...",10.01.2024 16:43,10.01.2024 23:59


In [11]:
weather_df.head()

,time,temperature_2m,relative_humidity_2m,wind_speed_10m
0,01/12/23 00:00:00.000000,0.0235,94.346542,5.600286
1,01/12/23 01:00:00.000000,0.5235,90.332314,7.895416
2,01/12/23 02:00:00.000000,0.5235,90.332314,6.369050
3,01/12/23 03:00:00.000000,0.5235,90.332314,4.510787
4,01/12/23 04:00:00.000000,0.4735,91.992157,3.319036
